<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       IVSM Banking Customer Churn Embed BYOM
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>Import the required libraries</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [7]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd

import teradataml as tdml
import getpass
from teradataml import in_schema
from teradataml import DecisionForest, XGBoost, TrainTestSplit, DecisionForestPredict, XGBoostPredict, SentimentExtractor, ColumnTransformer, ScaleFit, OneHotEncodingFit
from teradataml import ColumnSummary, AutoML, AutoClassifier
from teradataml import RoundColumns, ClassificationEvaluator, ROC
from teradataml import (
    DataFrame
)
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from nyoka import xgboost_to_pmml
from teradataml import save_byom,list_byom,retrieve_byom,delete_byom,PMMLPredict

In [2]:
tdml.configure.val_install_location = "VAL_USER"

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Initiate a connection to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [4]:
# Change host and/or username as needed
engine = tdml.create_context(
    host='host.docker.internal',
    username='demo_user',
    password=getpass.getpass(prompt='Password:'),
    logmech="TD2",
    encryptdata=True,
    database='demo_user'
)

Password: ···········


<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>1.1 Confirmation for functions</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Before starting let us confirm that the required functions are installed.</p>

In [8]:
from IPython.display import display, Markdown

df_check= DataFrame.from_query('''select count(*) as cnt from dbc.tablesV where databasename = 'ivsm';''')
if df_check.get_values()[0][0] >= 10:
    print('Functions are installed, please continue.')
else:
    print('Functions are not installed, please go to Instalization notebook before proceeding further')
    display(Markdown("[Initialization Notebook](./IVSM_Banking_Customer_Churn_Model_Install.ipynb)"))

Functions are installed, please continue.


In [9]:
df = tdml.DataFrame('semantic_search_results')
df[df['reference_txt'] == 'Negative or Abusive comment']

target_id,reference_id,target_txt,reference_txt,similarity
15686974,2,"Dear Team, I have constantly found your services to be reliable, but recently I had an unpleasant experience. A bank charge was levied on my account of which I was uninformed. I am disappointed as I value proper communication and understanding my finances. I'd appreciate if you could look into the matter, please rectify the error, if any. Thank you for your prompt attention. Best regards.",Negative or Abusive comment,0.6223594063463517
15781586,2,"Dear Team, While I generally enjoy your services, I recently encountered an upsetting issue. My mobile banking app continuously crashes, hindering my transaction processes. Also, the support team's response time is far slower than desirable, causing further inconvenience. I respectfully request attention is given to improve these components to ensure a smoother experience. Sincerely, [Your Name]",Negative or Abusive comment,0.566561626694944
15572158,2,"Dear Team, I'm reaching out to voice a concern regarding the mobile app. Lately, I've been confronting issues while transferring funds. It frequently logs out, hindering smooth transactions. Despite its user-friendly interface and great features, this persistent problem is quite frustrating. I believe your team's technical expertise can quickly address and rectify this glitch. Looking forward to an improved banking experience. Thank you.",Negative or Abusive comment,0.5700201231424351
15707962,2,"Subject: Issue with Online Banking Services Dear Sir/Madam, I am writing to express concerns regarding the frequent downtime of your online banking service. It often disrupts my transactions causing substantial inconvenience. While I appreciate the seamless experience your bank generally offers, this recurring issue is affecting my banking experience. I kindly request you to address this problem at the earliest. Thank you for your attention in this matter. Yours sincerely, [Your Name]",Negative or Abusive comment,0.5619092568149936
15760216,2,"Subject: Need for Improvement in Customer Service I am writing to express my concern with the recent service I received at your branch. The length of time I spent waiting was quite long, not to mention questioning the turbulent online transactions I've experienced. I believe in my long-standing relationship with your bank, hence, I hope you will address these issues promptly to enhance your customer service. Thanks.",Negative or Abusive comment,0.585195147596718
15724402,2,"I am dismayed by the recent handling of my account. Without pre-notice, my personal account got abruptly frozen causing immense inconvenience. It has been 72 hours without resolution. This indicates a lack of effective customer service. My bills have been delayed, causing potential impacts on my credit score. Highly unacceptable! Please resolve at the earliest.",Negative or Abusive comment,0.5858671429958989
15800287,2,"I am extremely disappointed with the service received at your bank. My checking account was erroneously overcharged by $200, causing multiple payment declines and inconveniences. To make matters worse, the customer service response was slow and unhelpful. It's disheartening to see such a lack of responsibility from a reputed banking institution. This experience has seriously impacted my trust in your organization. Immediate resolution is expected.",Negative or Abusive comment,0.5874785065209565
15605804,2,"I am displeased with the service at your bank lately. There have been multiple unexpected charges on my account, and all attempts to resolve the issue with your customer service team have been futile. The incompetence and disregard have seriously eroded my faith in your institution. I urge you to promptly rectify these errors and improve your customer service approach.",Negative or Abusive comment,0.6138876789140879
15582129,2,"Subject: Unexpected Transaction Fees Encountered Dear Sir/Madam, I hope this message finds you well. I recently noticed 

In [10]:
df[df['reference_txt'] == 'Positive and Upbeat comment']

target_id,reference_id,target_txt,reference_txt,similarity
15601026,1,"Subject: Positive Feedback on Customer Service, Request for Improvement in Mobile App Usability Dear Team, I must commend your staff's excellent customer service always polite and helpful. However, I've been encountering technical glitches with your mobile banking app. Errors appear during transactions and it often crashes, disrupting my banking activities. I believe enhancing your app's performance would greatly improve customer experience. Thank you.",Positive and Upbeat comment,0.5679251013530743
15808189,1,"Dear Team, I appreciate your consistent service, but lately, I have been encountering delayed response times on your mobile app. It's affecting my ability to make time-sensitive transactions. I believe this may be an issue with the app's update. Keeping the high standard of your bank in mind, I am confident that this will be addressed at the earliest. Thank you.",Positive and Upbeat comment,0.5786657458372437
15623346,1,"Dear Bank, I always appreciate your great service, but recently I encountered an issue with your mobile app. It keeps crashing each time I try to transfer funds, which is quite inconvenient. As a loyal customer, I expect the app to be technically efficient and user-friendly. I trust you will look into this matter promptly. Thank you.",Positive and Upbeat comment,0.5711684266320165
15672082,1,"Subject: Positive Feedback with Minor Issue I cherish your banking institution for its exemplary services. I recently encountered a little hiccup with the mobile app's functionality, where it keeps crashing upon trying to access my account details. The professionalism and efficiency usually exhibited make me confident that this issue I'm experiencing will be resolved promptly. Your continuous dedication to customer satisfaction never goes unnoticed.",Positive and Upbeat comment,0.6504360190900562
15758685,1,"Dear Team, I've always appreciated the excellent service you provide, but sadly, recently I faced an unexpected issue. My online account shows a debit transaction I never authorized. This comes as a surprise as I've enjoyed accurate and prompt services till now. Rest assured, I trust you will look into this issue immediately and rectify it, thereby restoring my confidence in your esteemed banking institution. Best Regards",Positive and Upbeat comment,0.5654982570582617
15595221,1,"Dear Sir/Madam, I have been a loyal customer for several years and greatly appreciate your efficient service. However, I recently encountered an issue where my mobile banking app abruptly crashes, hindering smooth transactions. I'm positive that your technical team can rectify this problem swiftly. Looking forward to an uninterrupted banking experience. Best regards.",Positive and Upbeat comment,0.588930054527305
15762418,1,"Dear Sir/Madam, I am writing to bring to your attention a matter that could significantly improve my banking experience. I noticed that your online platform can become considerably slow during peak hours, causing slight inconveniences. I commend your esteemed institution for its wonderful service over the years, and I feel this constructive feedback will only enhance your customer satisfaction. Thank you for your prompt attention to this matter. Best Regards.",Positive and Upbeat comment,0.6628720860082561
15784148,1,"Dear Sir/Madam, I've always appreciated your excellent customer service; however, recently, I experienced a slight hiccup. Yesterday, I tried to make an online purchase but my card got declined, despite having sufficient funds in my account. Also, the transaction history hasn't updated in a few days. I trust in your ability to resolve this issue soon. Thank you!",Positive and Upbeat comment,0.5986100863876718
15686436,1,"Subject: Constructive Feedback Regarding Your Banking Service I have been a customer at your bank for several years and generally, I've had a great experience. However, lately I've noticed the app has been crashin

<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage.</p>
<p style = 'font-size:16px;font-family:Arial'><b><i>*Please scroll down to the end of the notebook for detailed column descriptions of the dataset.</i></b></p>

In [11]:
customer_churn = tdml.DataFrame(in_schema('demo_user','bank_churn'))

In [12]:
new_df = customer_churn.merge(df[['target_id','reference_txt']], on='customerid = target_id', how='inner')
new_df

Unnamed: 0,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,target_id,reference_txt
27,15681697,Rueda,508,France,Male,31,8,72541.48,1,1,0,129803.08,0,15681697,Positive and Upbeat comment
36,15629617,Cook,572,Spain,Male,23,2,126873.52,1,0,1,67040.12,0,15629617,Positive and Upbeat comment
42,15642345,Y?,714,Germany,Female,49,4,93059.34,1,1,0,7571.51,1,15642345,Negative or Abusive comment
44,15769959,Ajuluchukwu,597,France,Female,53,4,88381.21,1,1,0,69384.71,1,15769959,Positive and Upbeat comment
52,15809906,Mitchell,558,Germany,Male,26,1,148853.29,2,1,1,24411.02,0,15809906,Negative or Abusive comment
57,15585901,Johnson,717,Spain,Male,35,1,0.0,3,0,0,174770.14,1,15585901,Negative or Abusive comment
8,15771270,North,635,France,Female,27,8,127471.56,1,1,1,152916.05,1,15771270,Negative or Abusive comment
10,15773242,Chukwuhaenye,621,France,Male,32,1,0.0,2,1,1,168779.47,0,15773242,Negative or Abusive comment
27,15681697,Rueda,508,France,Male,31,8,72541.48,1,1,0,129803.08,0,15681697,Positive and Upbeat comment
29,15667265,Cavenagh,729,France,Male,39,4,121404.64,1,1,1,159618.17,0,15667265,Negative or Abusive comment


In [13]:
new_df = new_df.drop('target_id',axis=1)

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Transformation</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>Define Column Categories</b> </p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Specifies the target variable and categorizes input columns into numeric, categorical, binary, and identifier groups for preprocessing and modeling.<br>

In [14]:
target_variable = "Exited"
numeric_columns = ["Age", "Balance", "CreditScore", "EstimatedSalary", "Tenure"]
categorical_columns = ["Gender", "Geography", "reference_txt", "NumOfProducts"]
binary_columns = ["HasCrCard", "IsActiveMember"]
id_column = ["CustomerId"]

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>ScaleFit()</b> function outputs statistics to input to ScaleTransform() function, which scales specified input DataFrame columns.<br>

In [15]:
fit1 = ScaleFit(data=new_df,
                target_columns=numeric_columns,
                scale_method="USTD",
                miss_value="KEEP",
                global_scale=False,
                multiplier="1")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>OneHotEncodingFit </b>outputs a table of attributes and categorical values to input to OneHotEncodingTransform which encodes them as one-hot numeric vectors.</p>

In [16]:
fit2 = OneHotEncodingFit(data=new_df,
                         is_input_dense=True,
                         approach="auto",
                         target_column=categorical_columns[0:3],
                         category_counts=[2,3,2])

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The <b>ColumnTransformer</b> function transforms the entire dataset in a single operation. You only need
to provide the FIT tables to the function, and the function runs all transformations that you require in a
single operation. Running all the it table transformations together in one-go gives approx. 30% performance improvement over running each transformation sequentially.</p>

In [17]:
new_table = ColumnTransformer(input_data=new_df,
                             onehotencoding_fit_data=fit2.result,
                             scale_fit_data=fit1.output).result

In [18]:
new_table=new_table[['CustomerId', 'Age', 'Balance', 'CreditScore', 'EstimatedSalary', 'Exited', 'HasCrCard', 'IsActiveMember',
                     'NumOfProducts', 'Tenure', 'Gender_0', 'Gender_1', 'Geography_0', 'Geography_1', 'Geography_2',
                     'reference_txt_0', 'reference_txt_1']]

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>3. Train-Test Split</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The <b>TrainTestSplit()</b> function divides the dataset into train and test subsets to be used for evaluating machine learning models and validation processes.<br>
80% is used for Training and 20% for validation.</p>

In [19]:
TrainTestSplit_out = TrainTestSplit(data = new_table,
                                    id_column='CustomerId',
                                    train_size=0.80,
                                    test_size=0.20,
                                    seed=3432)

In [20]:
TrainTestSplit_out.result.head()

TD_IsTrainRow,CustomerId,Age,Balance,CreditScore,EstimatedSalary,Exited,HasCrCard,IsActiveMember,NumOfProducts,Tenure,Gender_0,Gender_1,Geography_0,Geography_1,Geography_2,reference_txt_0,reference_txt_1
0,15791944,-0.6834124923509931,1.6194851649943567,0.4902341606236718,0.2809189669654197,1,1,0,3,0.7122855400071041,0,1,1,0,0,1,0
0,15584229,-1.6021148719117007,0.7803790192969021,0.22486204311174243,1.0328442833004756,0,1,1,1,1.4092381231843099,1,0,0,1,0,0,1
0,15721793,1.153992266770422,0.7802708771344405,-1.418403761481359,-1.335477718681342,0,1,1,1,0.7122855400071041,1,0,0,1,0,1,0
0,15752463,-0.989646618871229,0.8780146169353371,1.806888128279014,1.5673837477346662,0,0,1,1,-0.3331433347587045,1,0,0,0,1,1,0
0,15684147,0.4394459715565384,0.4285099924052654,0.29630838244187724,-0.36080899197884736,0,1,1,1,0.015332956829898384,0,1,1,0,0,1,0
0,15749401,2.1747726885045418,-1.2382178944926896,0.37796134167631706,-0.4376118260032969,1,1,1,3,1.4092381231843099,0,1,1,0,0,1,0
0,15813067,0.6436020559033624,0.5568628501238594,-2.214520114017147,-0.9992251261953323,1,1,0,1,-0.6816196263473073,1,0,0,1,0,1,0
0,15702175,-0.989646618871229,1.1828447347208384,1.0822181150733605,-1.2476226201990577,0,1,1,2,-0.3331433347587045,1,0,1,0,0,0,1
0,15585533,-0.27510032365734527,1.1678416383263217,0.30651500234618223,1.258680974242341,0,0,1,1,0.36380924841850126,0,1,1,0,0,1,0
0,15567486,0.23528988720971453,-1.2382178944926896,-0.15278289334754172,1.1199910754724838,0,1,1,2,-0.3331433347587045,1,0,0,0,1,1,0


In [21]:
df_train = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 1].drop(['TD_IsTrainRow'], axis = 1)
df_test = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 0].drop(['TD_IsTrainRow'], axis = 1)

print("Training Set = " + str(df_train.shape[0]) + ". Testing Set = " + str(df_test.shape[0]))

Training Set = 2400. Testing Set = 600


In [22]:
df_test.head()

CustomerId,Age,Balance,CreditScore,EstimatedSalary,Exited,HasCrCard,IsActiveMember,NumOfProducts,Tenure,Gender_0,Gender_1,Geography_0,Geography_1,Geography_2,reference_txt_0,reference_txt_1
15567446,0.03113380286289062,0.5789354146598962,-0.030303454495881996,-1.2121563750022677,0,1,1,1,1.4092381231843099,0,1,0,1,0,1,0
15568044,-0.785490534524405,-1.2382178944926896,-1.4388170012899688,-1.6432335837869425,0,1,1,2,0.7122855400071041,1,0,1,0,0,0,1
15568240,-0.887568576697817,0.018590611006606236,-1.6021229197588485,0.8086655977231725,0,0,1,2,1.7577144147729127,1,0,0,1,0,1,0
15568240,-0.887568576697817,0.018590611006606236,-1.6021229197588485,0.8086655977231725,0,0,1,2,1.7577144147729127,1,0,0,1,0,1,0
15568506,-0.785490534524405,-0.1431322470665054,-1.2755110828210892,1.0722268815854068,0,1,1,2,1.7577144147729127,1,0,0,1,0,0,1
15568834,-1.193802703218053,0.8045512355775993,0.5004407805279768,-1.267691880379289,0,0,0,2,0.36380924841850126,0,1,0,0,1,0,1
15568487,-0.3771783658307572,0.7913406280988232,0.6433334591882465,-0.5409579201155942,0,1,1,1,0.7122855400071041,0,1,1,0,0,1,0
15567486,0.23528988720971453,-1.2382178944926896,-0.15278289334754172,1.1199910754724838,0,1,1,2,-0.3331433347587045,1,0,0,0,1,1,0
15567446,0.03113380286289062,0.5789354146598962,-0.030303454495881996,-1.2121563750022677,0,1,1,1,1.4092381231843099,0,1,0,1,0,1,0
15567078,-1.193802703218053,-0.1600216424454045,1.4292431918197297,-0.3641490722156825,0,1,1,1,1.060761831595707,1,0,1,0,0,1,0


In [23]:
tdml.copy_to_sql(df_train, table_name = 'clean_data_train', if_exists = 'replace')
tdml.copy_to_sql(df_test, table_name = 'clean_data_test', if_exists = 'replace')

In [24]:
df_train = tdml.DataFrame(in_schema('demo_user','clean_data_train'))

In [25]:
df_test = tdml.DataFrame(in_schema('demo_user','clean_data_test'))

<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>3.1 Split Features and Target</b> </p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Separates feature columns and target labels for both training and test datasets, keeping CustomerId for reference and including encoded categorical and semantic features.</p>



In [26]:
df_train_features = df_train[['CustomerId', 'Age', 'Balance', 'CreditScore', 'EstimatedSalary', 
                              'HasCrCard', 'IsActiveMember', 'NumOfProducts','Tenure', 
                              'Gender_0', 'Gender_1', 'Geography_0', 'Geography_1', 
                              'Geography_2', 'reference_txt_0','reference_txt_1']]

df_train_target = df_train[['CustomerId', 'Exited']]
df_test_features = df_test[['CustomerId', 'Age', 'Balance', 'CreditScore', 'EstimatedSalary', 
                              'HasCrCard', 'IsActiveMember', 'NumOfProducts','Tenure', 
                              'Gender_0', 'Gender_1', 'Geography_0', 'Geography_1', 
                              'Geography_2', 'reference_txt_0','reference_txt_1']]

df_test_target = df_test[['CustomerId', 'Exited']]

In [27]:
tdml.copy_to_sql(df_train_features, table_name = 'xgb_train_features', if_exists = 'replace', schema_name = 'demo_user')
tdml.copy_to_sql(df_train_target, table_name = 'xgb_train_target', if_exists = 'replace', schema_name = 'demo_user')
tdml.copy_to_sql(df_test_features, table_name = 'xgb_test_features', if_exists = 'replace', schema_name = 'demo_user')
tdml.copy_to_sql(df_test_target, table_name = 'xgb_test_target', if_exists = 'replace', schema_name = 'demo_user')

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'> <b>4.Grant Access to ModelOps</b> </p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Grants SELECT permissions on training, test, and clean data tables to the modelops role, allowing model deployment processes to access the data.

In [28]:
SQL = ['''grant select on demo_user.xgb_train_features to modelops with grant option;''',
       '''grant select on demo_user.xgb_train_target to modelops with grant option;''',
       '''grant select on demo_user.xgb_test_features to modelops with grant option;''',
       '''grant select on demo_user.xgb_test_target to modelops with grant option;''',
       '''grant select on demo_user.clean_data_train to modelops with grant option;''',
       '''grant select on demo_user.clean_data_test to modelops with grant option;'''       
      ]

for i in SQL:
    try:
        tdml.execute_sql(i)
    except:
        True

In [29]:
train_pdf = df_train.to_pandas(all_rows=True)

features = cols = ['Age', 'Balance', 'CreditScore', 'EstimatedSalary', 'HasCrCard', 'IsActiveMember', 'NumOfProducts',
                   'Tenure', 'Gender_0', 'Gender_1', 'Geography_0', 'Geography_1', 'Geography_2', 'reference_txt_0',
                   'reference_txt_1']
target = "Exited"

# split data into X and y
X_train = train_pdf[features]
y_train = train_pdf[target]

model = Pipeline([('xgb', XGBClassifier(n_estimators=5, max_depth=10))])

model.fit(X_train, y_train)
#database = 'modelops'

Pipeline(steps=[('xgb',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=10, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=5,
                               n_jobs=None, num_parallel

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial'><b>5. Convert the model to PMML</b></p>
<p style = 'font-size:16px;font-family:Arial'>You can use the sklearn2pmml or the nyoka python libraries to convert to pmml. The nyoka is a python only package and so it is preferable.</p>

In [30]:
xgboost_to_pmml(
    pipeline=model, 
    col_names=cols, 
    target_name='Exited', 
    pmml_f_name="xgb_model.pmml")

In [31]:
tdml.configure.byom_install_location = "mldb"

In [32]:
try:
    save_byom("xgb_model",
              "xgb_model.pmml",
              "byom_models",
              additional_columns={},
              schema_name='modelops'
             )
except:
    delete_byom(model_id="xgb_model", table_name="byom_models", schema_name = 'modelops')
    save_byom("xgb_model",
              "xgb_model.pmml",
              "byom_models",
              additional_columns={},
              schema_name='modelops'
    )

Model is deleted.
Model is saved.


In [33]:
list_byom(table_name="byom_models", schema_name="modelops")

                              model
model_id                           
xgb_model  b'3C3F786D6C20766572...'


In [34]:
result = PMMLPredict(
    modeldata = retrieve_byom("xgb_model", "byom_models", schema_name="modelops"),
    newdata = df_test,
    accumulate = ['CustomerId'],
    overwrite_cached_models = '*',
)

print(result.show_query())

result.result

SELECT * FROM "mldb".PMMLPredict(
	ON "demo_user"."clean_data_test" AS InputTable
	PARTITION BY ANY 
	ON (select model_id,model from "DEMO_USER"."ml__filter__1746635246703385") AS ModelTable
	DIMENSION
	USING
	Accumulate('CustomerId')
	OverwriteCachedModel('*')
) as sqlmr


CustomerId,prediction,json_report
15568487,0,"{""probability_0"":0.8732029814111136,""probability_1"":0.1267970185888864,""predicted_Exited"":0}"
15569892,0,"{""probability_0"":0.8580923541145362,""probability_1"":0.14190764588546378,""predicted_Exited"":0}"
15571756,0,"{""probability_0"":0.885070089967748,""probability_1"":0.11492991003225193,""predicted_Exited"":0}"
15572626,0,"{""probability_0"":0.8918206231389475,""probability_1"":0.10817937686105249,""predicted_Exited"":0}"
15574910,0,"{""probability_0"":0.5211449028799425,""probability_1"":0.47885509712005747,""predicted_Exited"":0}"
15575438,0,"{""probability_0"":0.8726378873843766,""probability_1"":0.12736211261562339,""predicted_Exited"":0}"
15567078,0,"{""probability_0"":0.8477844647303625,""probability_1"":0.15221553526963755,""predicted_Exited"":0}"
15567446,0,"{""probability_0"":0.870261205306686,""probability_1"":0.1297387946933139,""predicted_Exited"":0}"
15567446,0,"{""probability_0"":0.870261205306686,""probability_1"":0.1297387946933139,""predicted_Exited"":0}"
15568044,0,"{""probability_0"":0.8930280970197778,""probability_1"":0.10697190298022222,""predicted_Exited"":0}"


<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'> <b>6. Clean up </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will remove the context.</p>

In [35]:
tdml.remove_context()

True

<hr style="height:1px;border:none;">
<b style = 'font-size:18px;font-family:Arial'>Dataset:</b>

- `Unnamed`: Unnamed
- `CustomerId`: Customer ID
- `Surname`: Surname
- `CreditScore`: Credit score
- `Geography`: Country (Germany / France / Spain)
- `Gender`: Gender (Female / Male)
- `Age`: Age
- `Tenure`: No of years the customer has been associated with the bank
- `Balance`: Balance
- `NumOfProducts`: No of bank products used
- `HasCrCard`: Credit card status (0 = No, 1 = Yes)
- `IsActiveMember`: Active membership status (0 = No, 1 = Yes)
- `EstimatedSalary`: Estimated salary
- `Exited`: Abandoned or not? (0 = No, 1 = Yes)

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>ScaleFit reference: <a href = 'https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/FEATURE-ENGINEERING-TRANSFORM-functions/ScaleFit'>here</a></li>
    <li>OneHotEncodingFit reference: <a href = 'https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/FEATURE-ENGINEERING-TRANSFORM-functions/OneHotEncodingFit'>here</a></li>
    <li>ColumnTransformer reference: <a href = 'https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/FEATURE-ENGINEERING-TRANSFORM-functions/ColumnTransformer'>here</a></li>
    <li>TrainTestSplit reference: <a href = 'https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/MODEL-EVALUATION-functions/TrainTestSplit'>here</a></li>
    <li>PMMLPredict reference: <a href = 'https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20/teradataml-Bring-Your-Own-Analytics/PMMLPredict'>here</a></li>
</ul>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>